<a href="https://colab.research.google.com/github/dbradul/ds/blob/master/am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How airmap slice polygon?

For https://api.airmap.com/elevation/v1/ele/carpet?points=28.8,33.8,28.81,33.81 we get 39x37 matrix. Possibly it is square in meters or some metric system

## Prepare calculation functions...

In [219]:
import math
import folium
from numpy import arange
from pprint import pprint as pp
import itertools


def get_steps(lat1, lon1, lat2, lon2, step_meters=25):
    # direct calc
    latitude_degrees_per_meter = 1 / 111320

    # depends on lat
    longitude_degrees_per_meter = 1 / \
    (111320 * abs(math.cos(math.radians((lat1 + lat2) / 2))))

    step_lat = step_meters * latitude_degrees_per_meter
    step_lon = step_meters * longitude_degrees_per_meter
    return (step_lat, step_lon)


def generate_squares(lat1, lon1, lat2, lon2, step_lat, step_lon):
    squares = []
    for sub_lat1 in arange(float(lat1), float(lat2), float(step_lat)):
        row = []
        for sub_lon1 in arange(float(lon1), float(lon2), float(step_lon)):
            sub_lat2 = sub_lat1 + step_lat
            sub_lon2 = sub_lon1 + step_lon
            if sub_lat2 <= lat2 and sub_lon2 <= lon2:
                square = [(sub_lat1, sub_lon1), (sub_lat2, sub_lon2)]
                row.append(square)

        squares.append(row)
    return squares



## Lets measure number of squares
Egypt (near equator)

In [ ]:
lat1, lon1 = 28.8, 33.8
lat2, lon2 = 28.81, 33.81
step_meters = 30 # magicaly
[step_lat, step_lon] = get_steps(lat1, lon1, lat2, lon2, step_meters)
squares = generate_squares(lat1, lon1, lat2, lon2, step_lat, step_lon)
print("lat", len(squares))
print("lon",len(squares[0]))


lat 38
lon 32


### Hm, lets build a map

In [ ]:


m = folium.Map(location=[(lat1 + lat2) / 2, (lon1 + lon2) / 2], zoom_start=19)

for row in squares:
    for square in row:
        sub_lat1 = square[0][0]
        sub_lon1 = square[0][1]
        sub_lat2 = square[1][0]
        sub_lon2 = square[1][1]

        folium.Rectangle(
            bounds=[(sub_lat1, sub_lon1), (sub_lat2, sub_lon2)],
            color='red',
            fill=False,
        ).add_to(m)
folium.Rectangle(
    bounds=[(lat1, lon1), (lat2, lon2)],
    color='yellow',
    fill=False,
).add_to(m)

m


In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth's surface
    given their latitude and longitude in decimal degrees.

    :param lat1: Latitude of the first point
    :param lon1: Longitude of the first point
    :param lat2: Latitude of the second point
    :param lon2: Longitude of the second point
    :return: Distance in meters
    """
    # Radius of the Earth in kilometers
    earth_radius = 6371

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = earth_radius * c

    return distance * 1000

hd = haversine_distance


In [ ]:
# curl https://api.airmap.com/elevation/v1/ele/carpet?points=5.0,5.0,5.01,5.01 | jq '{ "row_count": .data.carpet | length, "column_count": .data.carpet[0] | length }'
{
  "row_count": 37,
  "column_count": 37
}

{'row_count': 37, 'column_count': 37}

In [ ]:
lat_dist = hd(5.0, 5.0, 5.01, 5.0)
lon_dist = hd(5.0, 5.0, 5.0, 5.01)
lat_dist, lon_dist

(1111.9492664456066, 1107.7179637694392)

In [ ]:
lat_step_size = lat_dist / 37
lon_step_size = lon_dist / 37
lat_step_size, lon_step_size

(30.052682876908285, 29.938323345119976)

In [ ]:
lat_dist = hd(48.8, 38.8, 48.81, 38.8)
lon_dist = hd(48.8, 38.8, 48.8, 38.81)
lat_dist, lon_dist

(1111.9492664462257, 732.4292614685385)

## How AirMap works

AM повертає різну розмірність для різних координат через округлення та те, які координати зберігає у себе в БД

Приклад: ми запрошуємо мапу висот для коодинат: -35.13,148.74,-35.12,148.75

Дельта по лат і лон фіксована: 0.01

AM повертає мапу для трішки інших координат (мабуть для найближчих, які є у нього в БД):

In [ ]:
# curl https://api.airmap.com/elevation/v1/ele/carpet?points=-35.13,148.74,-35.12,148.75 | jq '{ "lat1": .data.bounds.sw[0], "lon1": .data.bounds.sw[1], "lat2": .data.bounds.ne[0], "lon2": .data.bounds.ne[1] } | {lat1, lon1, lat2, lon2, delta_lat: (.lat2-.lat1), delta_lon: (.lon2-.lon1)}'
{
  "lat1": -35.13027777777778,
  "lon1": 148.74,
  "lat2": -35.11972222222222,
  "lon2": 148.75,
  "delta_lat": 0.01055555555555543,
  "delta_lon": 0.009999999999990905
}


{'lat1': -35.13027777777778,
 'lon1': 148.74,
 'lat2': -35.11972222222222,
 'lon2': 148.75,
 'delta_lat': 0.01055555555555543,
 'delta_lon': 0.009999999999990905}

Дельта по lat достатньо сильно відрізняється від 0.01: 0.01055

Через це розмірніть по lat та lon буде відрізнятись:

In [ ]:
# curl https://api.airmap.com/elevation/v1/ele/carpet?points=-35.13,148.74,-35.12,148.75 | jq '{ "lat_count": .data.carpet | length, "lon_count": .data.carpet[0] | length }'
{
  "lat_count": 39,
  "lon_count": 37
}

{'lat_count': 39, 'lon_count': 37}

Розмірність він обчислює так:
1/3600 - це 1 arc-секунда

In [ ]:
# curl https://api.airmap.com/elevation/v1/ele/carpet?points=-35.13,148.74,-35.12,148.75 | jq '{ "lat1": .data.bounds.sw[0], "lon1": .data.bounds.sw[1], "lat2": .data.bounds.ne[0], "lon2": .data.bounds.ne[1] } | {lat1, lon1, lat2, lon2, delta_lat: (.lat2-.lat1), delta_lon: (.lon2-.lon1)} | {lat_count_calc: (.delta_lat/(1/3600)+1), lon_count_calc: (.delta_lon/(1/3600)+1)}'
{
  "lat_count_calc": 38.999999999999545,
  "lon_count_calc": 36.99999999996726
}

{'lat_count_calc': 38.999999999999545, 'lon_count_calc': 36.99999999996726}

Тобто фактична та підрахована розмірність співпадать для будь-яких 2х координат:


In [ ]:
# curl https://api.airmap.com/elevation/v1/ele/carpet?points=10.10,20.20,10.12,20.22 | jq '{ "lat1": .data.bounds.sw[0], "lon1": .data.bounds.sw[1], "lat2": .data.bounds.ne[0], "lon2": .data.bounds.ne[1], "lat_count": .data.carpet | length, "lon_count": .data.carpet[0] | length } | {lat_count, lon_count, lat_count_calc: ((.lat2-.lat1)/(1/3600)+1), lon_count_calc: ((.lon2-.lon1)/(1/3600)+1)}'
{
  "lat_count": 73,
  "lon_count": 73,
  "lat_count_calc": 72.99999999999847,
  "lon_count_calc": 72.99999999999847
}

{'lat_count': 73,
 'lon_count': 73,
 'lat_count_calc': 72.99999999999847,
 'lon_count_calc': 72.99999999999847}

## AM grid generation
Fixed degree delta for lat and long leads to fixed linear delta for lat, but variable for long. As a result square tile in terms of degrees generates trapecial shape linear tile and its subtiles.

It is displayed as a rectangular, not as trapezoid on a map, b/c of Mercator projection.



In [142]:
def generate_grid(lat1, lon1, lat2, lon2):
    # Fixed step size: 1 arcsecond
    grid_step = (1/3600)

    # rounding to get rid of float crazy fractions
    N_lat = math.ceil(round(lat2 - lat1, 4)/grid_step) + 1
    N_lon = math.ceil(round(lon2 - lon1, 4)/grid_step) + 1

    # Create a 2D grid
    grid = []

    # Fill the grid with values
    for i in range(N_lat):
        row = []
        for j in range(N_lon):
            lat_value = (lat1 + i * grid_step)
            lon_value = (lon1 + j * grid_step)
            row.append((lat_value, lon_value))
        grid.append(row)

    return grid


def validate_grid(grid):
    grid_step = (1/3600)
    sw_lat = grid[0][0][0]
    sw_lon = grid[0][0][1]
    ne_lat = grid[-1][-1][0]
    ne_lon = grid[-1][-1][1]
    print(sw_lat, sw_lon)
    print(ne_lat, ne_lon)
    assert math.isclose(sw_lat + (len(grid)-1)    * grid_step, ne_lat)
    assert math.isclose(sw_lon + (len(grid[0])-1) * grid_step, ne_lon)


# Donetsk
# lat1, lon1, lat2, lon2 = 48.0, 37.80, 48.01, 37.81

# Chernihiv
# lat1, lon1, lat2, lon2 = 51.50, 31.28, 51.51, 31.29

# Odesa
# lat1, lon1, lat2, lon2 = 46.48, 30.74, 46.485, 30.745

# # Zaporizhia
# lat1, lon1, lat2, lon2 = 47.82, 35.16, 47.82, 35.16 # one point
# lat1, lon1, lat2, lon2 = 47.82, 35.16, 47.82, 35.17 # 1d region
# lat1, lon1, lat2, lon2 = 47.82, 35.16, 47.83, 35.17 # 2d region

# Here we return 37*37, but AM returns 37*38  b/c it modifies lon1 a little bit: 35.16 -> 35.15972
lat1, lon1, lat2, lon2 = 47.82, 35.16, 47.83, 35.169 # 2d region


# Generate the gradient grid
grid = generate_grid(lat1, lon1, lat2, lon2)

# Validate grid
validate_grid(grid)

# Print the generated grid
grid[0][:10], grid[-1][-10:]


47.82 35.16
47.83 35.16916666666666


([(47.82, 35.16),
  (47.82, 35.16027777777777),
  (47.82, 35.160555555555554),
  (47.82, 35.16083333333333),
  (47.82, 35.161111111111104),
  (47.82, 35.16138888888889),
  (47.82, 35.16166666666666),
  (47.82, 35.161944444444444),
  (47.82, 35.16222222222222),
  (47.82, 35.162499999999994)],
 [(47.83, 35.166666666666664),
  (47.83, 35.16694444444444),
  (47.83, 35.16722222222222),
  (47.83, 35.1675),
  (47.83, 35.16777777777777),
  (47.83, 35.168055555555554),
  (47.83, 35.16833333333333),
  (47.83, 35.168611111111105),
  (47.83, 35.16888888888889),
  (47.83, 35.16916666666666)])

In [ ]:
m = folium.Map(location=[(lat1 + lat2) / 2, (lon1 + lon2) / 2], zoom_start=16)

for i in range(len(grid)-1):
    for j in range(len(grid[i])-1):
        sub_lat1 = grid[i][j][0]
        sub_lon1 = grid[i][j][1]
        sub_lat2 = grid[i+1][j+1][0]
        sub_lon2 = grid[i+1][j+1][1]

        folium.Rectangle(
            bounds=[(sub_lat1, sub_lon1), (sub_lat2, sub_lon2)],
            color='red',
            fill=False,
        ).add_to(m)
folium.Rectangle(
    bounds=[(lat1, lon1), (lat2, lon2)],
    color='yellow',
    fill=False,
).add_to(m)

m

## **misc** precision issue

In [ ]:
import math

grid_step = (1/3600)
print(grid_step)
lat1, lat2 =  30.2, 30.21
N_lat = math.ceil((lat2 - lat1) / grid_step) + 1
N_lat2 = math.ceil(round(lat2 - lat1, 4) / grid_step) + 1

print(N_lat, lat1 + (grid_step*N_lat))
print(N_lat2, lat1 + (grid_step*N_lat2))

0.0002777777777777778
38 30.210555555555555
37 30.210277777777776


In [ ]:
m = folium.Map(location=[(lat1 + lat2) / 2, (lon1 + lon2) / 2], zoom_start=16)

for i in range(len(grid)-1):
    for j in range(len(grid[i])-1):
        sub_lat1 = grid[i][j][0]
        sub_lon1 = grid[i][j][1]
        sub_lat2 = grid[i+1][j+1][0]
        sub_lon2 = grid[i+1][j+1][1]

        folium.Rectangle(
            bounds=[(sub_lat1, sub_lon1), (sub_lat2, sub_lon2)],
            color='red',
            fill=False,
        ).add_to(m)
folium.Rectangle(
    bounds=[(lat1, lon1), (lat2, lon2)],
    color='yellow',
    fill=False,
).add_to(m)

m

## Verification (autotest)

In [232]:
import math
import requests
from typing import List, Optional
from pydantic import BaseModel


class Bounds(BaseModel):
    sw: List[float]
    ne: List[float]

class Stats(BaseModel):
    max: int
    min: int
    avg: float

class Data(BaseModel):
    bounds: Bounds
    stats: Stats
    carpet: List[List[int]]

class Response(BaseModel):
    status: str
    data: Data


ARC_SEC = 1 / 3600
tiles = [
    (51.00, 31.00, 51.01, 31.01),
    (51.00, 31.00, 51.01, 31.02),
    (51.00, 31.00, 51.02, 31.01),
    (51.00, 31.00, 51.02, 31.02),

    (51.00, 31.00, 51.00, 31.01), # 1d row
    (51.00, 31.00, 51.01, 31.00), # 1d column
    (51.00, 31.00, 51.00, 31.00), # 1point

    (51.82, 30.16, 51.83, 30.161), # gradual change
    (51.82, 30.16, 51.83, 30.162),
    (51.82, 30.16, 51.83, 30.163),
    (51.82, 30.16, 51.83, 30.164),
    (51.82, 30.16, 51.83, 30.165),
    (51.82, 30.16, 51.83, 30.166),
    (51.82, 30.16, 51.83, 30.167),
    (51.82, 30.16, 51.83, 30.168),
    (51.82, 30.16, 51.83, 30.169),

    (51.50, 31.28, 51.51, 31.29), # misc
    (51.82, 30.16, 51.83, 30.17),
    (50.99, 31.00, 51.02, 31.02),
    (50.11, 30.14, 50.12, 30.15),
    (50.13, 30.16, 50.14, 30.17),
    (50.18, 30.11, 50.19, 30.12),
    (50.23, 30.24, 50.24, 30.25),
    # (47.82, 35.16, 47.83, 35.17),
]

def kr_build_url(lat1, lon1, lat2, lon2):
    PORT = 8085
    # BASE_URL = f"http://localhost:{PORT}"
    BASE_URL = "https://c4f8-213-231-6-236.ngrok-free.app"
    url = BASE_URL + f"/elevation/area?latitude1={lat1}&longitude1={lon1}&latitude2={lat2}&longitude2={lon2}"
    return url

def am_build_url(lat1, lon1, lat2, lon2):
    BASE_URL = "https://api.airmap.com/elevation/v1/ele/carpet?"
    url = BASE_URL + f"points={lat1},{lon1},{lat2},{lon2}"
    return url

def flatten (m):
    return [item for row in m for item in row]

def fetch_data(url_builder, lat1, lon1, lat2, lon2):
    url = url_builder(lat1, lon1, lat2, lon2)
    response = requests.get(url)
    assert response.status_code == 200
    obj = Response(**response.json())
    return obj


def test_bounds(lat1, lon1, lat2, lon2):
    print("TEST_FORMAT:", lat1, lon1, lat2, lon2)
    kr_data = fetch_data(kr_build_url, lat1, lon1, lat2, lon2)
    am_data = fetch_data(am_build_url, lat1, lon1, lat2, lon2)

    resp_lat1 = kr_data.data.bounds.sw[0]
    resp_lon1 = kr_data.data.bounds.sw[1]
    resp_lat2 = kr_data.data.bounds.ne[0]
    resp_lon2 = kr_data.data.bounds.ne[1]

    assert kr_data.status == "success"
    assert math.isclose(resp_lat1 + (len(kr_data.data.carpet)    - 1) * ARC_SEC, resp_lat2)
    assert math.isclose(resp_lon1 + (len(kr_data.data.carpet[0]) - 1) * ARC_SEC, resp_lon2)

    # pp((dict(kr_data), dict(am_data)))
    assert math.isclose(kr_data.data.bounds.sw[0], am_data.data.bounds.sw[0])
    assert math.isclose(kr_data.data.bounds.sw[1], am_data.data.bounds.sw[1])
    assert math.isclose(kr_data.data.bounds.ne[0], am_data.data.bounds.ne[0])
    assert math.isclose(kr_data.data.bounds.ne[1], am_data.data.bounds.ne[1])

    assert len(kr_data.data.carpet) == len(am_data.data.carpet)
    assert len(kr_data.data.carpet[0]) == len(am_data.data.carpet[0])


def test_content(lat1, lon1, lat2, lon2):
    print("TEST_CONTENT:", lat1, lon1, lat2, lon2)
    kr_data = fetch_data(kr_build_url, lat1, lon1, lat2, lon2)
    am_data = fetch_data(am_build_url, lat1, lon1, lat2, lon2)
    grid = generate_grid(lat1, lon1, lat2, lon2)

    kr_flt = flatten(kr_data.data.carpet)
    am_flt = flatten(am_data.data.carpet)

    max_of_two = max(max(kr_flt), max(am_flt))
    deltas = [
        (t[1]-t[2], abs(t[1]-t[2])/max_of_two)
        for t in zip(flatten(grid), kr_flt, am_flt)
    ]
    max_absolute = max([t[0] for t in deltas])
    max_relative = max([t[-1] for t in deltas])
    print("\tMax deltas (abs, norm):", max_absolute, "%.2f%%" % (max_relative*100))

    assert max_relative < 0.05


if __name__ == "__main__":
    for lat1, lon1, lat2, lon2 in tiles:
        test_bounds(lat1, lon1, lat2, lon2)

    for lat1, lon1, lat2, lon2 in tiles:
        test_content(lat1, lon1, lat2, lon2)

    print("PASSED!")

TEST_FORMAT: 51.0 31.0 51.01 31.01
TEST_FORMAT: 51.0 31.0 51.01 31.02
TEST_FORMAT: 51.0 31.0 51.02 31.01
TEST_FORMAT: 51.0 31.0 51.02 31.02
TEST_FORMAT: 51.0 31.0 51.0 31.01
TEST_FORMAT: 51.0 31.0 51.01 31.0
TEST_FORMAT: 51.0 31.0 51.0 31.0
TEST_FORMAT: 51.82 30.16 51.83 30.161
TEST_FORMAT: 51.82 30.16 51.83 30.162
TEST_FORMAT: 51.82 30.16 51.83 30.163
TEST_FORMAT: 51.82 30.16 51.83 30.164
TEST_FORMAT: 51.82 30.16 51.83 30.165
TEST_FORMAT: 51.82 30.16 51.83 30.166
TEST_FORMAT: 51.82 30.16 51.83 30.167
TEST_FORMAT: 51.82 30.16 51.83 30.168
TEST_FORMAT: 51.82 30.16 51.83 30.169
TEST_FORMAT: 51.5 31.28 51.51 31.29
TEST_FORMAT: 51.82 30.16 51.83 30.17
TEST_FORMAT: 50.99 31.0 51.02 31.02
TEST_FORMAT: 50.11 30.14 50.12 30.15
TEST_FORMAT: 50.13 30.16 50.14 30.17
TEST_FORMAT: 50.18 30.11 50.19 30.12
TEST_FORMAT: 50.23 30.24 50.24 30.25
TEST_CONTENT: 51.0 31.0 51.01 31.01
	Max deltas (abs, norm): 2 2.34%
TEST_CONTENT: 51.0 31.0 51.01 31.02
	Max deltas (abs, norm): 4 3.03%
TEST_CONTENT: 51.0 31.

## Verification (visual)

In [233]:
from folium.plugins import HeatMap

# FINDME
# lat1, lon1, lat2, lon2 = 51.20, 30.200, 51.21, 30.21
# lat1, lon1, lat2, lon2 = 51.82, 30.16, 51.83, 30.166
lat1, lon1, lat2, lon2 = 50.99, 30.995, 51.02, 31.02

kr_data = fetch_data(kr_build_url, lat1, lon1, lat2, lon2)
am_data = fetch_data(am_build_url, lat1, lon1, lat2, lon2)

print(kr_data.data.bounds.sw, kr_data.data.bounds.ne, len(kr_data.data.carpet), len(kr_data.data.carpet[0]))
print(am_data.data.bounds.sw, am_data.data.bounds.ne, len(am_data.data.carpet), len(am_data.data.carpet[0]))
print(kr_data.data.carpet[0][:10], kr_data.data.carpet[-1][-10:])
print(am_data.data.carpet[0][:10], am_data.data.carpet[-1][-10:])

[50.99, 30.995] [51.02, 31.02] 109 91
[50.99, 30.995] [51.02, 31.02] 109 91
[113, 113, 113, 113, 113, 113, 112, 112, 111, 111] [111, 112, 112, 113, 113, 113, 113, 114, 114, 113]
[113, 113, 113, 113, 113, 113, 112, 112, 111, 111] [111, 112, 112, 113, 113, 113, 113, 114, 114, 113]


In [234]:
grid = generate_grid(lat1, lon1, lat2, lon2)
print(len(grid), len(grid[0]))

109 91


In [244]:
# merge grid and heights
import random

kr_flt = flatten(kr_data.data.carpet)
am_flt = flatten(am_data.data.carpet)

# print(kr_flt)

max_of_two = max(max(kr_flt), max(am_flt))
print(max_of_two)

data = [
    # (pair[0][0], pair[0][1], random.random())
    (t[0][0], t[0][1], abs(t[1]-t[2])/max_of_two)
    for t in zip(flatten(grid), kr_flt, am_flt)
]
print(max_of_two, max([t[-1] for t in data]))
# pp(list(zip(kr_data.data.carpet[37], am_data.data.carpet[37])))
# pp((data[:], data[-10:]))
# data[-1] = (data[-1][0], data[-1][1], 1)
# print(max_of_two, max([t[-1] for t in data]))
pp((data[:10], data[-10:]))


132
132 0.030303030303030304
([(50.99, 30.995, 0.0),
  (50.99, 30.99527777777778, 0.0),
  (50.99, 30.995555555555555, 0.0),
  (50.99, 30.995833333333334, 0.0),
  (50.99, 30.996111111111112, 0.0),
  (50.99, 30.99638888888889, 0.0),
  (50.99, 30.996666666666666, 0.0),
  (50.99, 30.996944444444445, 0.0),
  (50.99, 30.997222222222224, 0.0),
  (50.99, 30.997500000000002, 0.0)],
 [(51.02, 31.017500000000002, 0.0),
  (51.02, 31.017777777777777, 0.0),
  (51.02, 31.018055555555556, 0.0),
  (51.02, 31.018333333333334, 0.0),
  (51.02, 31.018611111111113, 0.0),
  (51.02, 31.01888888888889, 0.0),
  (51.02, 31.019166666666667, 0.0),
  (51.02, 31.019444444444446, 0.0),
  (51.02, 31.019722222222224, 0.0),
  (51.02, 31.02, 0.0)])


In [242]:
m = folium.Map(location=[(lat1 + lat2) / 2, (lon1 + lon2) / 2], zoom_start=15.5)
HeatMap(data).add_to(m)
m